In [2]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.2'
spark_version = 'spark-3.1.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backpor

In [ ]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
# !wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Vine-Reviews-Analysis").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
url = "https://smartsanket-bucket.s3.us-east-2.amazonaws.com/amazon_reviews_us_Office_Products_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Office_Products_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   43081963|R18RVCKGH1SSI9|B001BM2MAC|     307809868|Scotch Cushion Wr...| Office Products|          5|            0|          0|   N|                Y|          Five Stars|      Great product.| 2015-08-31|
|         US|   10951564|R3L4L6LW1PUOFY|B00DZYEXPQ|      75004341|Dust-Off Compress...| Office Products|          5|    

In [5]:
# Create new DataFrame with total_votes grater than 20
total_votes_df = df.filter("total_votes > 20")
total_votes_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   11212836|R243UXMMSSZVS6|B00C2MBMNS|     934393421|Lion 2-Pocket Pla...| Office Products|          2|          254|        254|   N|                Y|      Disappointing!|We ordered about ...| 2015-08-31|
|         US|   23384746|R2P92EHR0S5GBB|B00C47U5OI|     712795023|24pcs Vitamin pil...| Office Products|          1|    

In [7]:
# Creating a new DataFrame with rows where helpful_votes/total_votes greater than 0.5
helpful_total_votes_df = total_votes_df.filter("helpful_votes/total_votes > 0.5")
# Creating a new column to check if helpful_votes/total_votes > 0.5 is true in the new DataFrame
helpful_total_votes_df = helpful_total_votes_df.withColumn("helpful_total_ratio", helpful_total_votes_df["helpful_votes"]/helpful_total_votes_df["total_votes"])
helpful_total_votes_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|helpful_total_ratio|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+-------------------+
|         US|   11212836|R243UXMMSSZVS6|B00C2MBMNS|     934393421|Lion 2-Pocket Pla...| Office Products|          2|          254|        254|   N|                Y|      Disappointing!|We ordered about ...| 2015-08-31|                1.0|
|         US|   23384746|R2P92EHR0S5GBB|

In [10]:
# Create new DataFrame by retrieving all rows where reviews are paid (Vine program)
paid_reviews_df = helpful_total_votes_df.filter("vine == 'Y'")
paid_reviews_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|helpful_total_ratio|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+-------------------+
|         US|   35360512|R3JIGR122X50ZV|B00ZI7MI96|     592756103|Scotch Thermal La...| Office Products|          5|         2280|       2314|   Y|                N|Here’s what’s REA...|You’re reading th...| 2015-08-21| 0.9853068280034573|
|         US|   50120605|R3KV8P8WLD2KKC|

In [11]:
# Create new DataFrame by retrieving all rows where reviews are not paid (not part of Vine program)
unpaid_reviews_df = helpful_total_votes_df.filter("vine == 'N'")
unpaid_reviews_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|helpful_total_ratio|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+-------------------+
|         US|   11212836|R243UXMMSSZVS6|B00C2MBMNS|     934393421|Lion 2-Pocket Pla...| Office Products|          2|          254|        254|   N|                Y|      Disappointing!|We ordered about ...| 2015-08-31|                1.0|
|         US|   23384746|R2P92EHR0S5GBB|

In [21]:
# Counting Total Reviews, Number of 5 Start Reviews, and Percentage of 5 Star Reviews in Paid (Vine) Program 
total_paid_reviews = paid_reviews_df.count()
paid_five_star = paid_reviews_df.filter("star_rating == 5").count()
percent_paid_five_star = (paid_five_star/total_paid_reviews) * 100

print(f"Total number of Paid Reviews: {total_paid_reviews}")
print(f"Number of Paid 5 Star Reviews: {paid_five_star}")
print(f"Percentage of Paid 5 Star Reviews: {percent_paid_five_star:.2f}%")

Total number of Paid Reviews: 926
Number of Paid 5 Star Reviews: 408
Percentage of Paid 5 Star Reviews: 44.06%


In [22]:
# Counting Total Reviews, Number of 5 Start Reviews, and Percentage of 5 Star Reviews in Unpaid (not Vine) Program 
total_unpaid_reviews = unpaid_reviews_df.count()
unpaid_five_star = unpaid_reviews_df.filter("star_rating == 5").count()
percent_unpaid_five_star = (unpaid_five_star/total_unpaid_reviews) * 100

print(f"Total number of Unpaid Reviews: {total_unpaid_reviews}")
print(f"Number of Unpaid 5 Star Reviews: {unpaid_five_star}")
print(f"Percentage of Unpaid 5 Star Reviews: {percent_unpaid_five_star:.2f}%")

Total number of Unpaid Reviews: 40990
Number of Unpaid 5 Star Reviews: 18087
Percentage of Unpaid 5 Star Reviews: 44.13%
